# FUNCIONES

In [1]:
import os
import numpy as np
import copy
from math import sqrt
from scipy.linalg import solve_triangular

### Creación matrices

In [2]:
def crea_matriz(renglones,columnas,maximo_valor,minimo_valor,entero=False):
    """
    Función de apoyo para genear matrices aleatorias
                            
    params: renglones       no. de renglones de la matriz
            columnas        no. de renglones de la matriz
            maximo_valor    valor máximo de las entradas de la matriz
            minimo_valor    valor mínimo de las entradas de la matriz
            entero          Indica si las entradas serán enteras (True) o no
            
    return: M               Matriz con numeros al azar
    """
    M=np.zeros((renglones, columnas))
    for i in range(renglones):
        for j in range(columnas):
            if entero:
                M[i][j]=(np.random.rand(1)*(maximo_valor+1-minimo_valor)+minimo_valor)//1
            else:
                M[i][j]=np.random.rand(1)*(maximo_valor-minimo_valor)+minimo_valor
    return M

### Factorización QR

In [3]:
def house(x):
    """
    Función que calcula la proyección de householder
    
    params: x       vector al que se le hará la reflexión householder
                    
    return: Beta    constante utilizada para obtener v
            v       vector que representa la reflexión de householder
    """
    m=len(x)
    norm_2_m=x[1:m].dot(np.transpose(x[1:m]))
    v=np.concatenate((1,x[1:m]), axis=None)
    Beta=0
    if (norm_2_m==0 and x[0]>=0):
        Beta=0
    elif (norm_2_m==0 and x[0]<0):
        Beta=2
    else:
        norm_x=np.sqrt(pow(x[0],2)+norm_2_m)
        if (x[0]<=0):
            v[0]=x[0]-norm_x
        else:
            v[0]=-norm_2_m/(x[0]+norm_x)
        Beta=2*pow(v[0],2)/(norm_2_m+pow(v[0],2))
        v=v/v[0]
    return Beta, v

In [9]:
def factorizacion_QR(A):
    """
    Función que genera una matriz que contendrá información escencial de las proyecciones householder
    (vectores v's) y componentes de la matriz triangular superior R, del estilo:
    [r11      r12      r13      r14    ]
    [v_2_(1)  r22      r23      r24    ]
    [v_3_(1)  v_3_(2)  r33      r34    ]
    [v_4_(1)  v_4_(2)  v_4_(3)  r44    ]
    [v_5_(1)  v_5_(2)  v_5_(3)  v_5_(4)]
    
    params: A      Matriz (mxn) de la que se desea obtner factorización QR
            
    return: A_r_v  Matriz (mxn) con la información escencial (es igual a la matriz R, pero en lugar de tener ceros
                   en la parte inferior, contiene info de los vectores householder que serán útiles para
                   futuros cálculos, que entre otros están el calcular la matriz ortonormal Q)
    """
    m=A.shape[0]
    n=A.shape[1]
    A_r_v=copy.copy(A)
    for j in range(n):
        beta, v=house(A_r_v[j:m,j])
        A_r_v[j:m,j:n]=A_r_v[j:m,j:n]-beta*(np.outer(v,v)@A_r_v[j:m,j:n])
        A_r_v[(j+1):m,j]=v[1:(m-j)]
    return A_r_v

In [10]:
def QT_C(A_r_v,C):
    """
    Función que calcula el producto matricial de Q_transpuesta por una matriz dada C
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: M       Matriz con numero al azar
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    QT_por_C=np.eye(m)
    for j in range(n-1,-1,-1):
        v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
        beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
        QT_por_C[j:m,j:m]=C[j:m,j:m]-beta*np.outer(v,v)@C[j:m,j:m]
    return QT_por_C

In [11]:
def Q_j(A_r_v,j):
    """
    Función que calcula la matriz Qj (en el proceso de obtención de factorización QR se van obteniendo n Qj's,
    que si se multiplican todas da por resultado Q=Q1*Q2*...*Qn)
                            
    params: A_r_v   Matriz (mxn) con la info escencial
            C       Matriz (mxp) (si se pasa por ejemplo C=Identidad (mxm) la funcion devolverá Q)

    return: Qj      Matriz Q de la j-esima iteración del proceso iterativo de factorización QR
    """
    m=A_r_v.shape[0]
    n=A_r_v.shape[1]
    Qj=np.eye(m)
    v=np.concatenate((1,A_r_v[(j+1):m,j]), axis=None)
    beta=2/(1+A_r_v[(j+1):m,j].dot(A_r_v[(j+1):m,j]))
    Qj[j:m,j:m]=np.eye(m-j)-beta*np.outer(v,v)
    return Qj

### Funciones para solución de Sistemas de Ecuaciones Lineales

In [56]:
def Solucion_SEL_QR_nxn(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema

    return: x   vector que satisface (Ax=b)
    """
    A_r_v=factorizacion_QR(A)
    m=A_r_v.shape[0]
    #Q=np.transpose(QT_C(A_r_v,np.eye(m)))
    #R=np.transpose(Q)@A
    n=A_r_v.shape[0]
    Q=np.eye(m)
    R=copy.copy(A)
    for j in range(m):
        Qj=Q_j(A_r_v,j)
        Q=Q@Qj
        R=Q_j(A_r_v,j)@R
    b_prima=np.transpose(Q)@b
    x = solve_triangular(R, np.transpose(Q)@b)
    return x

#### Eliminación por bloques

In [13]:
def bloques(A, b=False, n1=False, n2=False):
    """
    Esta es la función para la creación de bloques usando un arreglo de numpy
    
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
            n1  Numero de renglones que tendrá el 1er bloque
            n2  Numero de renglones que tendrá el 2do bloque
    
    return: A11 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            A12 Fraccion de la matriz dividida
            b1  Fraccion del vector dividido
            b2  Fraccion del vector dividido
    """

    # Primero definimos el n
    m,n = A.shape

    # Condiciones de A
    # Si no se dan los n deseados, se intentan hacer los bloques casi iguales
    if  not (n1&n2):
        n1 = n//2
        n2 = n - n1
    # Los bloques deben cumplir la condicion de tamaño
    elif n1+n1 != n:
        sys.exit('n1 + n2 debe ser igual a n')
    else:
        None

    # Condiciones de b
    if  b is False:
        b1 = None
        b2 = None
        print('condicion1')
    elif len(b) == m:
        b1 = b[:n1]
        b2 = b[n1:m]
    else:
        sys.exit('los renglones de A y b deben ser del mismo tamaño')

    A11 = A[:n1,:n1]
    A12 = A[:n1,n1:n]
    A21 = A[n1:m,:n1]
    A22 = A[n1:m,n1:n]

    return A11,A12,A21,A22,b1,b2

In [14]:
def eliminacion_bloques(A,b):
    """
    Función que obtiene la solución de un sistema de ecuaciones lineala (SEL) con n ecuaciones y n incognitas
            
    params: A   Matriz (nxn) que representa los coeficientas de las ecuaciones
            b   vector (nx1) constantes del sistema
    
    return: x1 Solucion al 1er sistema de ecuaciones obtenido con la división por bloques
            x2 Solucion al 2do sistema de ecuaciones obtenido con la división por bloques
    """
    if np.linalg.det(A)==0:
        sys.exit('A debe ser no singular')

    A11,A12,A21,A22,b1,b2 = bloques(A,b)

    if np.linalg.det(A11)==0:
        ys.exit('A11 debe ser no singular')

    ## 1. Calcular A11^{-1}A12 y A11^{-1}b1 teniendo cuidado en no calcular la inversa sino un sistema de ecuaciones lineales
    ## Aquí se debe usar el método QR una vez que esté desarrollado

    ## Definimos y = A11^{-1}b1, por tanto A11y=b1. Resolviendo el sistema anterior para 11y:
    y = Solucion_SEL_QR_nxn(A11,b1)
    #y = np.linalg.solve(A11,b1)

    ## Definimos Y = A11^{-1}A12
    Y = Solucion_SEL_QR_nxn(A11,A12)
    #Y = np.linalg.solve(A11,A12)

    ## 2. Calcular el complemento de Schur del bloque A11 en A. Calcular b_hat
    S = A22 - A21@Y
    b_h = b2 - A21@y

    ## 3. Resolver Sx2 = b_hat
    x2 = Solucion_SEL_QR_nxn(S,b_h)
    #x2 = np.linalg.solve(S,b_h)

    ## 4. Resolver A11x1 = b1-A12X2
    x1 = Solucion_SEL_QR_nxn(A11,b1-A12@x2)
    #x1 = np.linalg.solve(A11,b1-A12@x2)

    return np.concatenate((x1,x2), axis=0)

# EJEMPLOS

**Generación de matrices**

In [16]:
#Prueba también definiendo una matriz entrada por entrada, solo asegurate que sus entradas sean
#tipo flotantes/dobles (pues si son de tipo enteras arrastrarán errores de redondeo considerables)
m=3
n=3
A=np.array([[1, 2, 1], [2, 3, 2], [1, 2, 3]], dtype='d')
A

array([[1., 2., 1.],
       [2., 3., 2.],
       [1., 2., 3.]])

In [18]:
#Generar una matriz aleatoria de tamaño definido (renglones y columnas) y valores aleatorios comprendidos en un rango, pudiendo ser sus entradas de tipo enteros o dobles
m=5
n=3
A=np.round(crea_matriz(m,n,6,-6,False),2)
A

array([[ 2.49,  2.14, -1.71],
       [ 5.05,  0.01, -5.  ],
       [ 1.12,  4.02,  1.97],
       [ 0.31,  2.89,  5.06],
       [-4.96,  4.65,  0.41]])

**Ejemplos de factorización de una matriz A**

In [26]:
A_r_v = factorizacion_QR(A)
np.round(A_r_v,4)

array([[ 7.5931, -1.6181, -3.6568],
       [-0.9896,  6.9352,  2.1373],
       [-0.2195, -1.5111,  6.2957],
       [-0.0607, -0.9726, -0.0094],
       [ 0.972 , -0.311 ,  0.2636]])

In [27]:
#Obtención de Q con la función QT_C
Q=np.transpose(QT_C(A_r_v,np.eye(m)))
np.round(Q,4)

array([[ 0.3279,  0.6651,  0.1475,  0.0408, -0.6532],
       [ 0.6651,  0.3418, -0.146 , -0.0404,  0.6464],
       [ 0.1475, -0.146 ,  0.9676, -0.009 ,  0.1434],
       [ 0.0408, -0.0404, -0.009 ,  0.9975,  0.0397],
       [-0.6532,  0.6464,  0.1434,  0.0397,  0.3651]])

In [28]:
#Así obtenemos R
R=np.transpose(Q)@A
np.round(R,4)

array([[ 7.5931, -1.6181, -3.6568],
       [ 0.    ,  3.7291, -3.0734],
       [ 0.    ,  4.8448,  2.3973],
       [ 0.    ,  3.1183,  5.1783],
       [-0.    ,  0.9972, -1.4822]])

In [29]:
#Ejemplo ilustrativo de cómo obtener Q y R, visualizando cada iteración
Q=np.eye(m)
R=copy.copy(A)
for j in range(n):
    Qj=Q_j(A_r_v,j)
    print('\nQ',j,':\n',np.round(Qj,4))
    Q=Q@Qj
    R=Q_j(A_r_v,j)@R
    print('\nAplicando las Qjs a A por la izquierda para obtener R, iteracion ',j,':\n',np.round(R,4))

print('\n\n\nResultados finales:')
print('\nR es el resultado de multiplicar todas las Qjs a A\n',np.round(R,4))
print('\nQ es el resultado de multiplicar todas las Qjs y transponer:\n',np.round(Q,4))


Q 0 :
 [[ 0.3279  0.6651  0.1475  0.0408 -0.6532]
 [ 0.6651  0.3418 -0.146  -0.0404  0.6464]
 [ 0.1475 -0.146   0.9676 -0.009   0.1434]
 [ 0.0408 -0.0404 -0.009   0.9975  0.0397]
 [-0.6532  0.6464  0.1434  0.0397  0.3651]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  0 :
 [[ 7.5931 -1.6181 -3.6568]
 [ 0.      3.7291 -3.0734]
 [ 0.      4.8448  2.3973]
 [ 0.      3.1183  5.1783]
 [-0.      0.9972 -1.4822]]

Q 1 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      0.5377  0.6986  0.4496  0.1438]
 [ 0.      0.6986 -0.0557 -0.6795 -0.2173]
 [ 0.      0.4496 -0.6795  0.5627 -0.1399]
 [ 0.      0.1438 -0.2173 -0.1399  0.9553]]

Aplicando las Qjs a A por la izquierda para obtener R, iteracion  1 :
 [[ 7.5931 -1.6181 -3.6568]
 [ 0.      6.9352  2.1373]
 [ 0.     -0.     -5.4768]
 [ 0.     -0.      0.1102]
 [-0.     -0.     -3.103 ]]

Q 2 :
 [[ 1.      0.      0.      0.      0.    ]
 [ 0.      1.      0.      0.      0.    ]
 [ 0.      0.     -0.8699  0.0175 -0.4929]

In [31]:
#Se prueba un ejemplo más grande con 10^4 entradas
m=125
n=80
A=np.round(crea_matriz(m,n,10,-10,False),2)
A_r_v = factorizacion_QR(A)
A_r_v

array([[ 6.15033568e+01,  7.77385211e-01,  4.76226039e-01, ...,
        -4.52931375e-01, -7.81323370e+00, -2.08715762e+00],
       [-1.07089700e-01,  6.51779209e+01, -3.51895715e+00, ...,
        -6.11006753e+00,  1.06230650e+01,  2.80690950e+00],
       [ 1.34754539e-01, -4.52623106e-03,  6.37240288e+01, ...,
         9.19699119e+00, -8.38007003e+00, -3.03195907e+00],
       ...,
       [ 1.94843759e-02,  5.54718683e-02, -1.44156077e-02, ...,
        -1.78932287e-01, -1.64536367e-01,  7.96156686e-02],
       [ 9.95041794e-02, -1.05126168e-01, -1.53877002e-01, ...,
         8.19434479e-02, -4.65339962e-02,  5.03881923e-02],
       [ 3.16807029e-02,  7.41492130e-02, -1.19758714e-01, ...,
        -1.13059862e-01,  3.07917029e-02, -1.35085392e-01]])

**Solucion de SEL**

### A) Sistema con soluciones únicas

In [57]:
#Generamos un sistema de ecuaciones lineales
m=6
n=6
A=np.round(crea_matriz(m,n,6,-6,False),2)
b = np.round(crea_matriz(m,1,6,-6,False),2)
print("A:",A)
print("b:",b)

A: [[-0.07  3.27 -1.79  4.04 -5.38 -2.56]
 [ 5.29 -1.63  1.93  2.43  2.17  2.42]
 [ 4.76 -0.74  1.83  0.47 -5.62 -2.24]
 [ 0.39 -1.79 -1.99 -4.93 -4.45 -2.97]
 [-2.9   4.14 -2.59 -4.46 -2.28  0.46]
 [ 5.17 -5.7   0.13  2.46  2.95 -2.8 ]]
b: [[ 5.43]
 [ 5.92]
 [ 3.06]
 [ 5.  ]
 [ 1.93]
 [-4.59]]


##### Primero resolvamos el sistema de ecuaciones usando la paquetería de numpy para comparar.

In [58]:
np.linalg.solve(A,b)

array([[ 0.15637675],
       [-2.42679687],
       [-3.16977472],
       [ 0.98960498],
       [-2.86444165],
       [ 4.57266594]])

##### Ahora usemos la factorización QR

In [59]:
x=Solucion_SEL_QR_nxn(A,b)
x

array([[ 0.15637675],
       [-2.42679687],
       [-3.16977472],
       [ 0.98960498],
       [-2.86444165],
       [ 4.57266594]])

In [61]:
#Checamos que sea solucion (debe ser igual al vector b)
A@x

array([[ 5.43],
       [ 5.92],
       [ 3.06],
       [ 5.  ],
       [ 1.93],
       [-4.59]])

##### Por último usemos eliminación por bloques con QR

In [38]:
eliminacion_bloques(A,b)

array([[2.29471547],
       [1.66948307],
       [0.94052672],
       [0.6261769 ],
       [2.17306118],
       [0.30206387]])

### B) Sistema sin solución

### C) Sistema con infinitas soluciones